In [1]:
import numpy as np
import dlib    #To detect and localize facial landmarks 
import cv2
from threading import Thread
import threading
import imutils
from imutils import face_utils
from scipy.spatial import distance as dist
import pygame

ModuleNotFoundError: No module named 'dlib'

In [2]:
def sound_alarm():
    pygame.mixer.init()
    pygame.mixer.music.load("C:\\Users\\hp\\Desktop\\dataset\\Alarm.mp3")
    pygame.mixer.music.play()

In [3]:
def eye_aspect_ratio(eye):
    
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])   #vertical distance
    
    C = dist.euclidean(eye[0], eye[3])   #horizontal distance
    
    ear = (A + B) / (2.0 * C)
    
    return ear

In [4]:
'''ap = argparse.ArgumentParser()

ap.add_argument("-p", "--shape-predictor", required=True, help="path to facial landmark predictor")

ap.add_argument("-a", "--alarm", type=str, default="", help="path alarm .WAV file")

ap.add_argument("-w", "--webcam", type=int, default=0, help="index of webcam on system")

args = vars(ap.parse_args(args=[]))'''

'ap = argparse.ArgumentParser()\n\nap.add_argument("-p", "--shape-predictor", required=True, help="path to facial landmark predictor")\n\nap.add_argument("-a", "--alarm", type=str, default="", help="path alarm .WAV file")\n\nap.add_argument("-w", "--webcam", type=int, default=0, help="index of webcam on system")\n\nargs = vars(ap.parse_args(args=[]))'

In [5]:
EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 20

COUNTER = 0
ALARM_ON = False

In [6]:
predictor_path = 'C:\\Users\\hp\\Desktop\\dataset\\shape_predictor_68_face_landmarks.dat'
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)


In [7]:
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [ ]:
vs = cv2.VideoCapture(0)

while True:
    ret, frame = vs.read()
    if ret == False:
        print('Failed to capture frame from camera. Check camera index in cv2.VideoCapture(0) \n')
        break
   # cv2.imshow(frame)
    
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)
    
    for rect in rects:
        shape = predictor(gray, rect)    # determine the facial landmarks for the face region
        shape = face_utils.shape_to_np(shape) #converting to numpy array
        
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        
        ear = (leftEAR + rightEAR) / 2.0
        
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        
        
        if ear < EYE_AR_THRESH:
            COUNTER += 1
        
            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                if not ALARM_ON:
                    ALARM_ON = True
                    d=threading.Thread(target=sound_alarm)
                    d.setDaemon(True)
                    d.start()
                    
                cv2.putText(frame, "DROWSINESS ALERT!", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                
        else:
            COUNTER = 0
            ALARM_ON = False
            
        cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord("q"):
        break
        
cv2.destroyAllWindows()
vs.release()